
# Cyber Crime Data Analysis (Simplified)

This notebook analyses the cyber crime losses and complaints dataset using Python and pandas.  
The goal is to produce simple, easy‑to‑understand tables and charts showing the top five countries by losses and complaints for each year (2019–2024), along with basic trend and correlation analyses.  
All code is heavily commented so that non‑technical users can follow along.


## Load and coerce

In [2]:

import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image, display

# Load the datasets. These must not be modified directly.
df_threats = pd.read_csv('Global_Cybersecurity_Threats_2015-2024.csv')
df_indices = pd.read_csv('Cyber_security.csv')
df_loss = pd.read_csv('LossFromNetCrime.csv')

# Replace missing country names with a placeholder to avoid issues when sorting.
df_loss['Country'] = df_loss['Country'].fillna('Unknown')

# Convert all numeric columns to numbers (they may already be numbers, but this ensures consistency).
numeric_cols = [c for c in df_loss.columns if c != 'Country']
df_loss[numeric_cols] = df_loss[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Define the years we'll be working with.
years = [2019, 2020, 2021, 2022, 2023, 2024]

## Computing total lossess worldwide

In [20]:
def extract_year(col):
    try:
        return int(str(col).split("_")[0])
    except Exception:
        return None

loss_columns = sorted([c for c in df_loss.columns if str(c).lower().endswith("_losses")], key=extract_year)
comp_columns = sorted([c for c in df_loss.columns if str(c).lower().endswith("_complaints")], key=extract_year)

loss_totals = pd.DataFrame({
    "year": [extract_year(c) for c in loss_columns],
    "total_losses": [pd.to_numeric(df_loss[c], errors="coerce").sum() for c in loss_columns]
}).dropna(subset=["year"]).astype({"year": int})

comp_totals = pd.DataFrame({
    "year": [extract_year(c) for c in comp_columns],
    "total_complaints": [pd.to_numeric(df_loss[c], errors="coerce").sum() for c in comp_columns]
}).dropna(subset=["year"]).astype({"year": int})

totals = pd.merge(loss_totals, comp_totals, on="year", how="outer").sort_values("year")
totals.to_csv('Code_Output/totals.csv', index=False)
totals

,year,total_losses,total_complaints
0,2019,27686530359,2884094
1,2020,32898320639,5060074
2,2021,55139794808,5350493
3,2022,84187894180,5029126
4,2023,100502375119,5324045
5,2024,118626086516,5935856


In [21]:
# Save global total losses by year to CSV
totals[['year', 'total_losses']].to_csv('Code_Output/global_total_losses_by_year.csv', index=False)
# Optional: display the data in the notebook
totals[['year', 'total_losses']]


,year,total_losses
0,2019,27686530359
1,2020,32898320639
2,2021,55139794808
3,2022,84187894180
4,2023,100502375119
5,2024,118626086516


In [22]:
# Save global total complaints by year to CSV for PowerBI
totals[['year', 'total_complaints']].to_csv('Code_Output/global_total_complaints_by_year.csv', index=False)
totals[['year', 'total_complaints']]


,year,total_complaints
0,2019,2884094
1,2020,5060074
2,2021,5350493
3,2022,5029126
4,2023,5324045
5,2024,5935856


## Separate DataFrame for TOP5 Countries for Complaints(No. of Attacks)

In [23]:

# Create an empty DataFrame to hold the top five countries for complaints.
top5_complaints = pd.DataFrame()

# For each year, sort the data by the complaints column in descending order,
# pick the top five countries, and store their names in a new column.
for year in years:
    col_name = f'{year}_Complaints'
    # Sort by complaints and take the top five country names.
    top_countries = df_loss.sort_values(by=col_name, ascending=False)['Country'].head(5).values
    # Assign this list to the corresponding year column in the top5_complaints DataFrame.
    top5_complaints[str(year)] = top_countries

# Display the resulting top 5 complaints table.
top5_complaints


,2019,2020,2021,2022,2023,2024
0,CN,US,CN,US,US,US
1,US,CN,US,CN,CN,CN
2,GB,GB,GB,GB,GB,GB
3,FR,RU,RU,RU,FR,RU
4,RU,HK,HK,HK,RU,BR


## Separate DataFrame for TOP5 Countries for Losses(in USD)

In [7]:

# Create an empty DataFrame to hold the top five countries for losses.
top5_losses = pd.DataFrame()

# For each year, sort the data by the losses column in descending order,
# pick the top five countries, and store their names in a new column.
for year in years:
    col_name = f'{year}_Losses'
    # Sort by losses and take the top five country names.
    top_countries = df_loss.sort_values(by=col_name, ascending=False)['Country'].head(5).values
    # Assign this list to the corresponding year column in the top5_losses DataFrame.
    top5_losses[str(year)] = top_countries

# Display the resulting top 5 losses table.
top5_losses


,2019,2020,2021,2022,2023,2024
0,US,US,US,US,US,US
1,CN,CN,RU,RU,RU,CN
2,RU,DE,CN,CN,CN,RU
3,DE,RU,GB,GB,DE,GB
4,GB,GB,UA,SG,UA,DE


## Creating tables for each year 2019 to 2024 + save to .CSV

In [24]:
# Build Top 5 tables for each year (complaints and losses)
top5_complaints = pd.DataFrame()
top5_losses     = pd.DataFrame()

for year in [2019, 2020, 2021, 2022, 2023, 2024]:
    complaints_col = f'{year}_Complaints'
    losses_col     = f'{year}_Losses'

    top5_complaints[str(year)] = df_loss.sort_values(by=complaints_col, ascending=False)['Country'].head(5).values
    top5_losses[str(year)]     = df_loss.sort_values(by=losses_col,     ascending=False)['Country'].head(5).values

# Now save both tables – they are guaranteed to exist
top5_complaints.to_csv('Code_Output/top5_complaints_by_year.csv', index=False)
top5_losses.to_csv('Code_Output/top5_losses_by_year.csv', index=False)
print('Saved top5_complaints_by_year.csv and top5_losses_by_year.csv')


Saved top5_complaints_by_year.csv and top5_losses_by_year.csv


## Top 5 countries (based on the top 5 number of complaints each year)

In [25]:

# for each year showing the top five countries by complaints.
# Complaints are shown in millions to keep the numbers readable.

for year in years:
    col_name = f'{year}_Complaints'
    # Select the top five countries for this year.
    top5 = df_loss.sort_values(by=col_name, ascending=False).head(5)
    countries = top5['Country']
    values = top5[col_name] / 1_000_000  # convert to millions

In [26]:
# Save top 5 countries by complaints per year to CSV for PowerBI
top5_rows = []
for year in years:
    col_name = f'{year}_Complaints'
    top5 = df_loss.sort_values(by=col_name, ascending=False).head(5)
    for country, value in zip(top5['Country'], top5[col_name]):
        top5_rows.append({'Year': year, 'Country': country,
                          'Complaints_Millions': value / 1_000_000})
top5_df = pd.DataFrame(top5_rows)
top5_df.to_csv('Code_Output/top5_countries_by_complaints.csv', index=False)
top5_df  # optional display


,Year,Country,Complaints_Millions
0,2019,CN,0.449305
1,2019,US,0.428355
2,2019,GB,0.182883
3,2019,FR,0.160810
4,2019,RU,0.159459
5,2020,US,0.796395
6,2020,CN,0.705690
7,2020,GB,0.427367
8,2020,RU,0.293917
9,2020,HK,0.237997


## Table - total complaints and lossess across all countries year-by-year

In [28]:

# Calculate total complaints and losses across all countries for each year.
totals = []
for year in years:
    total_complaints = df_loss[f'{year}_Complaints'].sum()
    total_losses = df_loss[f'{year}_Losses'].sum()
    totals.append({'Year': year, 'Total_Complaints': total_complaints,
                   'Total_Losses': total_losses})

totals_df = pd.DataFrame(totals)

# Compute the change from the previous year (NaN for the first year).
totals_df['Complaints_Change'] = totals_df['Total_Complaints'].diff()
totals_df['Losses_Change'] = totals_df['Total_Losses'].diff()

# Display the totals and changes.
totals_df.to_csv('Code_Output/top5_countries_by_complaints.csv')
totals_df


,Year,Total_Complaints,Total_Losses,Complaints_Change,Losses_Change
0,2019,2884094,27686530359,NaN,NaN
1,2020,5060074,32898320639,2175980.0,5.211790e+09
2,2021,5350493,55139794808,290419.0,2.224147e+10
3,2022,5029126,84187894180,-321367.0,2.904810e+10
4,2023,5324045,100502375119,294919.0,1.631448e+10
5,2024,5935856,118626086516,611811.0,1.812371e+10


## Correlation between complaints and lossses for each year

In [29]:

# Compute the Pearson correlation between complaints and losses for each year.
correlations = {}
for year in years:
    complaints_col = f'{year}_Complaints'
    losses_col = f'{year}_Losses'
    corr_value = df_loss[complaints_col].corr(df_loss[losses_col])
    correlations[year] = corr_value

# Convert to a DataFrame for display
corr_df = pd.DataFrame(list(correlations.items()), columns=['Year', 'Correlation'])

# Display correlation values
corr_df

# Save correlation values to CSV for PowerBI
corr_df.to_csv('Code_Output/correlation_complaints_losses.csv', index=False)
corr_df  # optional display


,Year,Correlation
0,2019,0.918155
1,2020,0.920393
2,2021,0.882538
3,2022,0.928676
4,2023,0.893894
5,2024,0.924917


## Simple prediction (we check for AVG yearly change 2019-2024 and add it to 2024's value)

In [30]:

# We create a simple prediction for 2025 for the countries that frequently appear in the top five.
# The idea is to take the average yearly change in losses for each country and add it to 2024’s value.

# First, identify the union of all top countries from the top5 tables.
top_countries = set()
for year in years:
    # add top countries by complaints
    top_countries.update(df_loss.sort_values(by=f'{year}_Complaints', ascending=False)['Country'].head(5).tolist())
    # add top countries by losses
    top_countries.update(df_loss.sort_values(by=f'{year}_Losses', ascending=False)['Country'].head(5).tolist())

top_countries = sorted(top_countries)

# Prepare predictions
predictions = []
for country in top_countries:
    # Gather the past losses for this country
    losses = []
    for year in years:
        losses.append(df_loss.loc[df_loss['Country'] == country, f'{year}_Losses'].values[0])
    
    # Compute the year‑to‑year differences
    diffs = pd.Series(losses).diff().dropna()
    # If there is no variance (e.g., constant), set avg_diff to zero
    avg_diff = diffs.mean() if not diffs.empty else 0
    
    # Predicted 2025 losses = 2024 losses + average yearly difference
    predicted_loss_2025 = losses[-1] + avg_diff
    
    predictions.append({'Country': country, 'Losses_2024': losses[-1],
                         'Predicted_Losses_2025': predicted_loss_2025})

pred_df = pd.DataFrame(predictions)

# Display the predictions
pred_df.sort_values(by='Predicted_Losses_2025', ascending=False)

,Country,Losses_2024,Predicted_Losses_2025
9,US,14457011740,1.668787e+10
1,CN,11299124337,1.305255e+10
6,RU,11161782401,1.291315e+10
4,GB,6379113995,7.386454e+09
2,DE,5279486289,6.023500e+09
8,UA,5161336915,5.935474e+09
7,SG,4524535787,5.219159e+09
3,FR,4025659986,4.637240e+09
5,HK,2863308167,3.286647e+09
0,BR,2590565434,2.994372e+09


## Plot line chart + prediction for 2025

In [32]:
# Save actual and predicted losses by country and year to CSV for PowerBI
rows = []
for country in pred_df['Country']:
    # actual 2019–2024 losses (in billions)
    actual_losses = [df_loss.loc[df_loss['Country'] == country, f'{year}_Losses'].values[0] for year in years]
    actual_losses_b = [val / 1_000_000_000 for val in actual_losses]
    for year_val, val in zip(years, actual_losses_b):
        rows.append({'Country': country, 'Year': year_val,
                     'Losses_Billions': val, 'Type': 'Actual'})
    # predicted 2025 loss
    pred_2025_b = pred_df.loc[pred_df['Country'] == country,
                              'Predicted_Losses_2025'].values[0] / 1_000_000_000
    rows.append({'Country': country, 'Year': 2025,
                 'Losses_Billions': pred_2025_b, 'Type': 'Predicted'})
plot_df = pd.DataFrame(rows)
plot_df.to_csv('Code_Output/actual_vs_predicted_losses.csv', index=False)
plot_df


,Country,Year,Losses_Billions,Type
0,BR,2019,0.571530,Actual
1,BR,2020,0.749658,Actual
2,BR,2021,0.894516,Actual
3,BR,2022,1.644489,Actual
4,BR,2023,1.808031,Actual
...,...,...,...,...
65,US,2021,6.466619,Actual
66,US,2022,10.304393,Actual
67,US,2023,11.916782,Actual
68,US,2024,14.457012,Actual


## Prediction narrowed down to TOP countries

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1) Determine top countries by total losses and total complaints
years = [2019, 2020, 2021, 2022, 2023, 2024]

# Compute total losses and complaints across all years for each country
df_loss['Total_Losses']     = df_loss[[f'{y}_Losses'     for y in years]].sum(axis=1)
df_loss['Total_Complaints'] = df_loss[[f'{y}_Complaints' for y in years]].sum(axis=1)

# Extract top-5 lists based on these totals
top5_by_losses     = df_loss.nlargest(5, 'Total_Losses')['Country'].tolist()
top5_by_complaints = df_loss.nlargest(5, 'Total_Complaints')['Country'].tolist()

# Combine the two lists (remove duplicates)
selected_countries = sorted(set(top5_by_losses + top5_by_complaints))

# 2) Prepare predictions using a simple average-change method
predictions = []
for country in selected_countries:
    # Gather the six years of losses for this country
    losses = [df_loss.loc[df_loss['Country'] == country, f'{year}_Losses'].values[0] for year in years]
    
    # Compute year-to-year differences and their average
    diffs = np.diff(losses)  # array of [Loss2020 - Loss2019, Loss2021 - Loss2020, ...]
    avg_diff = diffs.mean() if len(diffs) > 0 else 0
    
    # Predict 2025 losses by adding the average difference to the 2024 value
    predicted_2025 = losses[-1] + avg_diff
    
    predictions.append({
        'Country': country,
        'Losses_2024': losses[-1],
        'Predicted_Losses_2025': predicted_2025
    })

pred_df = pd.DataFrame(predictions)

# 3) Save actual vs. predicted losses for each selected country to CSV for Power BI
rows = []
for country in selected_countries:
    # Actual 2019–2024 losses (converted to billions)
    actual_losses = [df_loss.loc[df_loss['Country'] == country, f'{year}_Losses'].values[0] for year in years]
    actual_losses_b = [val / 1e9 for val in actual_losses]
    for year_val, val in zip(years, actual_losses_b):
        rows.append({
            'Country': country,
            'Year': year_val,
            'Losses_Billions': val,
            'Type': 'Actual'
        })

    # Predicted 2025 loss (converted to billions)
    pred_2025_b = pred_df.loc[pred_df['Country'] == country, 'Predicted_Losses_2025'].values[0] / 1e9
    rows.append({
        'Country': country,
        'Year': 2025,
        'Losses_Billions': pred_2025_b,
        'Type': 'Predicted'
    })

plot_df = pd.DataFrame(rows)
plot_df.to_csv('Code_Output/actual_vs_predicted_selected_countries.csv', index=False)
plot_df  # optional: display the DataFrame in the notebook

,Country,Year,Losses_Billions,Type
0,CN,2019,2.532008,Actual
1,CN,2020,3.061084,Actual
2,CN,2021,5.067323,Actual
3,CN,2022,7.405961,Actual
4,CN,2023,8.997590,Actual
5,CN,2024,11.299124,Actual
6,CN,2025,13.052548,Predicted
7,DE,2019,1.559416,Actual
8,DE,2020,2.432408,Actual
9,DE,2021,2.226372,Actual
